In [1]:
import os
import itertools
from types import SimpleNamespace
import numpy
import matplotlib.pyplot as pyplot
from scipy.interpolate import make_interp_spline
from shapely import geometry
import LocalModule
import boxkit

In [2]:
DsetDirs, FileTags = LocalModule.case2_grid_convergence_dict()
Datasets = {}
for key in DsetDirs:
    Datasets[key] = LocalModule.read_datasets(DsetDirs[key], FileTags[key])

In [3]:
Blocks = {}
for key in Datasets:
    Blocks[key] = boxkit.mergeblocks(Datasets[key][0], ["dfun", "velx", "vely"]).blocklist[0]

GridScale = {"Case2/h40" : 1, "Case2/h80" : 2, "Case2/h160" : 4, "Case2/h320" : 8}

block = Blocks["Case2/h40"]
MeshCoarseX, MeshCoarseY = numpy.meshgrid(block.xrange("center")[block.xguard:block.nxb+block.xguard], 
                                          block.yrange("center")[block.yguard:block.nyb+block.yguard])

for block, scale in zip(list(Blocks.values())[1:], list(GridScale.values())[1:]):
    xmesh, ymesh = numpy.meshgrid(block.xrange("center")[block.xguard:block.nxb+block.xguard], 
                                  block.yrange("center")[block.yguard:block.nyb+block.yguard])
    
    MeshFineX, MeshFineY = [0., 0.]
    
    for yoff in range(scale):
        for xoff in range(scale):
            MeshFineX = MeshFineX + xmesh[yoff::scale,xoff::scale]/(scale**2)
            MeshFineY = MeshFineY + ymesh[yoff::scale,xoff::scale]/(scale**2)
    
    if not (numpy.abs(MeshFineX-MeshCoarseX) < 1e-13).all():
        print(f"{scale} MeshX mismatch")
    if not (numpy.abs(MeshFineY-MeshCoarseY) < 1e-13).all():
        print(f"{scale} MeshY mismatch")

In [4]:
GridReference = {}

scale = GridScale["Case2/h320"]
block = Blocks["Case2/h320"]
offset = int(scale/2)

for var in ["dfun", "velx", "vely"]:
    
    WorkData = block[var][0,block.yguard:block.nyb+block.yguard,
                            block.xguard:block.nxb+block.xguard]
    GridReference[var] = 0.
    for yoff in range(scale):
        for xoff in range(scale):
            GridReference[var] = GridReference[var] + WorkData[yoff::scale,xoff::scale]/(scale**2)

Error = []

for block, scale in zip(list(Blocks.values())[:-1], list(GridScale.values())[:-1]):
    
    GridData = {}
    BlockError = {}
    
    for var in ["dfun", "velx", "vely"]:
        WorkData = block[var][0,block.yguard:block.nyb+block.yguard,
                                block.xguard:block.nxb+block.xguard]
        
        if scale == 1:
            GridData[var] = WorkData
        else:
            
            GridData[var] = 0.       
            
            for yoff in range(scale):
                for xoff in range(scale):
                    GridData[var] = GridData[var] + WorkData[yoff::scale,xoff::scale]/(scale**2)
        
        BlockError[var] = numpy.max(numpy.abs(GridReference[var]-GridData[var]))
        
    Error.append(BlockError)

In [5]:
Error

[{'dfun': 0.11740511722563796,
  'velx': 0.23098055644395799,
  'vely': 0.3935689148356676},
 {'dfun': 0.07918093646452005,
  'velx': 0.1603827254987454,
  'vely': 0.3167824797273682},
 {'dfun': 0.030509918815824502,
  'velx': 0.13901148913688532,
  'vely': 0.2725374002064569}]